In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import nltk
import pandas as pd

data = pd.read_csv('/content/gdrive/My Drive/NLP/stance-data.csv')
data.head()

In [3]:
topic = "gay rights"
data = data[data.topic=="gay rights"][["post_text","label","word_count","words_pronom",
                                      "words_per_sen","words_over_6","pos_emo","neg_emo"]]
data.head()

,post_text,label,word_count,words_pronom,words_per_sen,words_over_6,pos_emo,neg_emo
1157,I believe it is a privilage for them . If they...,pro,28,9.33,0.00,0.00,10.71,35.71
1158,With marriage comes a lot of privileges . Taxe...,pro,35,11.67,2.86,0.00,11.43,11.43
1159,I do n't see why people have a problem with th...,pro,67,13.40,0.00,4.48,11.94,22.39
1160,[ *I 'm not homosexual but I do agree . ] Why ...,pro,151,12.58,1.32,0.66,16.56,23.18
1161,No-one should marry ! ! ! But gay people have ...,pro,15,7.50,0.00,0.00,0.00,6.67


In [4]:
np.random.seed(10)
shuffled_data = np.random.permutation(data)
np.shape(shuffled_data)

(1359, 8)

In [5]:
k = 5
test_size = len(shuffled_data)//k
# for i in range(k):
#   test = shuffled_data[i*test_size:(i+1)*test_size]
# index_test = i
test_data = shuffled_data[:test_size]
train_data = shuffled_data[test_size:]
print(np.shape(test_data))

(271, 8)


In [0]:
# Training
from sklearn.feature_extraction.text import CountVectorizer
#t0 = time()
num_features = 100
cv = CountVectorizer(stop_words='english', ngram_range=(1,3), 
                     max_features=num_features)
X_train = cv.fit_transform(train_data[:,0])
X_test = cv.fit_transform(test_data[:,0])

y_train = train_data[:,1]
y_test = test_data[:,1]

In [7]:
from collections import Counter
Counter(y_train)

Counter({'con': 392, 'pro': 696})

In [39]:
from scipy.sparse import hstack
X_train = hstack((X_train, train_data[:,2:].astype(float)))
X_test = hstack((X_test, test_data[:,2:].astype(float)))
print(np.shape(X_train))


(1088, 106)


In [0]:
target_names = ['pro', 'con']

In [0]:
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

In [0]:
def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."


# #############################################################################
# Benchmark classifiers
def benchmark(clf, print_report=True, print_cm=True, print_top10=True):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

#         if print_top10 and feature_names is not None:
#             print("top 10 keywords per class:")
#             #print(np.shape(clf.coef_[1]),clf.coef_[1])
#             #for i, label in enumerate(target_names):
#             top10_idx = np.argsort(clf.coef_[0])[-10:]
#             top10_wrd = [feature_names[i] for i in top10_idx]
#             print(trim(" %s" % ( " ".join(top10_wrd))))
#         print()

    if print_report:
        print("classification report:")
        print(metrics.classification_report(y_test, pred,
                                            target_names=target_names))

    if print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_test, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time

In [41]:
benchmark(MultinomialNB(alpha=.01))

________________________________________________________________________________
Training: 
MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
train time: 0.007s
test time:  0.001s
accuracy:   0.613
dimensionality: 106
density: 1.000000
classification report:
              precision    recall  f1-score   support

         pro       0.43      0.31      0.36        95
         con       0.67      0.78      0.72       176

    accuracy                           0.61       271
   macro avg       0.55      0.54      0.54       271
weighted avg       0.59      0.61      0.59       271

confusion matrix:
[[ 29  66]
 [ 39 137]]



('MultinomialNB',
 0.6125461254612546,
 0.007231235504150391,
 0.0008368492126464844)

In [47]:
benchmark(BernoulliNB(alpha=.01))

________________________________________________________________________________
Training: 
BernoulliNB(alpha=0.01, binarize=0.0, class_prior=None, fit_prior=True)
train time: 0.011s
test time:  0.001s
accuracy:   0.528
dimensionality: 106
density: 1.000000
classification report:
              precision    recall  f1-score   support

         pro       0.34      0.36      0.35        95
         con       0.64      0.62      0.63       176

    accuracy                           0.53       271
   macro avg       0.49      0.49      0.49       271
weighted avg       0.53      0.53      0.53       271

confusion matrix:
[[ 34  61]
 [ 67 109]]



('BernoulliNB',
 0.5276752767527675,
 0.011360883712768555,
 0.0009305477142333984)

In [50]:
benchmark(ComplementNB(alpha=.01))

________________________________________________________________________________
Training: 
ComplementNB(alpha=0.01, class_prior=None, fit_prior=True, norm=False)
train time: 0.008s
test time:  0.000s
accuracy:   0.594
dimensionality: 106
density: 1.000000
classification report:
              precision    recall  f1-score   support

         pro       0.42      0.39      0.40        95
         con       0.68      0.70      0.69       176

    accuracy                           0.59       271
   macro avg       0.55      0.55      0.55       271
weighted avg       0.59      0.59      0.59       271

confusion matrix:
[[ 37  58]
 [ 52 124]]



('ComplementNB',
 0.5940959409594095,
 0.008152961730957031,
 0.0004899501800537109)

In [44]:
benchmark(LinearSVC(penalty='l1', dual=False,tol=1e-3))

________________________________________________________________________________
Training: 
LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
          verbose=0)
train time: 0.022s
test time:  0.000s
accuracy:   0.579
dimensionality: 106
density: 0.990566
classification report:
              precision    recall  f1-score   support

         pro       0.36      0.26      0.30        95
         con       0.65      0.75      0.70       176

    accuracy                           0.58       271
   macro avg       0.51      0.51      0.50       271
weighted avg       0.55      0.58      0.56       271

confusion matrix:
[[ 25  70]
 [ 44 132]]



('LinearSVC', 0.5793357933579336, 0.022356271743774414, 0.00044035911560058594)

In [45]:
benchmark(LinearSVC(penalty='l2', dual=False,tol=1e-3))

________________________________________________________________________________
Training: 
LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.001,
          verbose=0)
train time: 0.013s
test time:  0.001s
accuracy:   0.568
dimensionality: 106
density: 1.000000
classification report:
              precision    recall  f1-score   support

         pro       0.35      0.26      0.30        95
         con       0.65      0.73      0.69       176

    accuracy                           0.57       271
   macro avg       0.50      0.50      0.49       271
weighted avg       0.54      0.57      0.55       271

confusion matrix:
[[ 25  70]
 [ 47 129]]



('LinearSVC', 0.5682656826568265, 0.012603044509887695, 0.0006315708160400391)

In [0]:
feature_names = cv.get_feature_names()

In [0]:
np.shape(feature_names)

(100,)

In [0]:
feature_names[:10]

['10',
 '10 20',
 '10 20 years',
 '10 million',
 '10 million baby',
 '10 years',
 '10 years greater',
 '100',
 '100 effective',
 '100 effective dont']

In [0]:
np.argsort(feature_names)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [0]:
np.argsort(feature_names)[-10:]

array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [0]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Olivier Grisel <olivier.grisel@ensta.org>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Lars Buitinck
# License: BSD 3 clause
import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

In [0]:
target_names = ['pro', 'con']

In [0]:
def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."


# #############################################################################
# Benchmark classifiers
def benchmark(clf, print_report=True, print_cm=True, print_top10=True):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

        if print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            #print(np.shape(clf.coef_[1]),clf.coef_[1])
            #for i, label in enumerate(target_names):
            top10_idx = np.argsort(clf.coef_[0])[-10:]
            top10_wrd = [feature_names[i] for i in top10_idx]
            print(trim(" %s" % ( " ".join(top10_wrd))))
        print()

    if print_report:
        print("classification report:")
        print(metrics.classification_report(y_test, pred,
                                            target_names=target_names))

    if print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_test, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time

In [0]:
benchmark(MultinomialNB(alpha=.01))

________________________________________________________________________________
Training: 
MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
train time: 0.006s
test time:  0.000s
accuracy:   0.520
dimensionality: 100
density: 1.000000
top 10 keywords per class:
 saying thing want let reason couples homosexuality man did need

classification report:
              precision    recall  f1-score   support

         pro       0.31      0.31      0.31        95
         con       0.63      0.64      0.63       176

    accuracy                           0.52       271
   macro avg       0.47      0.47      0.47       271
weighted avg       0.52      0.52      0.52       271

confusion matrix:
[[ 29  66]
 [ 64 112]]



('MultinomialNB',
 0.5202952029520295,
 0.005521059036254883,
 0.00022912025451660156)

In [0]:
clf = LinearSVC(penalty='l1', dual=False,tol=1e-3)
clf.fit(X_train, y_train)
# pred = clf.predict(X_test)
# score = metrics.accur
np.shape(clf.coef_)

(1, 100)

In [0]:
np.shape(clf.coef_)

(1, 100)

In [0]:
from sklearn.svm import LinearSVC
benchmark(LinearSVC(penalty='l1', dual=False,tol=1e-3))

________________________________________________________________________________
Training: 
LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
          verbose=0)
train time: 0.009s
test time:  0.000s
accuracy:   0.557
dimensionality: 100
density: 0.970000
top 10 keywords per class:


IndexError: ignored

## Method2